In [1]:
import random
random.seed(1000)
import ast
import os
import collections
from importlib import reload
import pandas as pd
import numpy as np
np.random.seed(1000)
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from matplotlib import cm
from matplotlib.colors import LightSource
import scipy.stats

In [2]:
os.getcwd()

'/Users/subhrasishchakraborty'

In [61]:
os.chdir('/Users/subhrasishchakraborty/Desktop')
calibration_data = pd.read_csv('/Users/subhrasishchakraborty/Desktop/calibration_data_acc_leeds.csv')
validation_data = pd.read_csv('/Users/subhrasishchakraborty/Desktop/validation_data_acc_leeds.csv')
os.chdir('/Users/subhrasishchakraborty/Desktop/lumbar-conformal-main/scripts')

In [62]:
import conformal_demo

In [63]:
#alphas = np.array([0.05,0.10,0.15,0.2,0.25])
alphas = np.array([0.05,0.06,0.07,0.08,0.09,0.1,0.11,0.12,0.13,0.14,0.15])

In [64]:
cal_labels = calibration_data["cal_labels"]
val_labels = validation_data["val_labels"]

In [66]:
validation_data

,Unnamed: 0,X1,X2,X3,val_labels
0,3,2.185390e-04,0.096059,0.903723,3
1,4,9.586234e-05,0.090080,0.909825,3
2,5,1.656744e-05,0.054663,0.945321,3
3,7,9.631474e-04,0.131337,0.867700,3
4,8,1.031014e-01,0.155926,0.740972,3
...,...,...,...,...,...
1110,1900,3.515550e-02,0.056836,0.908009,3
1111,1903,1.284894e-05,0.165176,0.834812,3
1112,1904,6.099199e-16,0.224010,0.775990,2
1113,1905,6.371939e-02,0.199236,0.737045,2


In [67]:
cal_scores = calibration_data[calibration_data.columns[1:4]]
val_scores = validation_data[validation_data.columns[1:4]]
val_scores

,X1,X2,X3
0,2.185390e-04,0.096059,0.903723
1,9.586234e-05,0.090080,0.909825
2,1.656744e-05,0.054663,0.945321
3,9.631474e-04,0.131337,0.867700
4,1.031014e-01,0.155926,0.740972
...,...,...,...
1110,3.515550e-02,0.056836,0.908009
1111,1.284894e-05,0.165176,0.834812
1112,6.099199e-16,0.224010,0.775990
1113,6.371939e-02,0.199236,0.737045


In [68]:
# cal_scores = cal_scores.to_numpy()
# val_scores = val_scores.to_numpy()
#num_trials = 10

In [71]:
alpha_aps_overall_coverage = collections.defaultdict(list)
alpha_aps_overall_size = collections.defaultdict(list)

In [72]:
cal_labels = cal_labels - 1
val_labels = val_labels - 1

In [73]:
num_trials = 5
for alpha in alphas:
    aps_overall_coverage = []
    aps_overall_size = []
    
    for trial in range(num_trials):
        aps_qhat = conformal_demo.get_qhat_ordinal_aps(
            conformal_demo.ordinal_aps_prediction,
            np.copy(cal_scores),
            np.copy(cal_labels),
            alpha)
        aps_pred_sets = conformal_demo.ordinal_aps_prediction(np.copy(val_scores), aps_qhat)
        aps_overall_coverage.append(aps_pred_sets[np.arange(val_scores.shape[0]), val_labels].sum() / val_labels.shape[0])
        aps_overall_size.append(aps_pred_sets.sum(1).mean())
        
    alpha_aps_overall_coverage[alpha] = aps_overall_coverage
    alpha_aps_overall_size[alpha] = aps_overall_size
    

In [74]:
aps_agg_coverage = np.array(list(alpha_aps_overall_coverage.values()))
aps_agg_size = np.array(list(alpha_aps_overall_size.values()))

In [75]:
agg_aps_cvg = []
for x in aps_agg_coverage:
    agg_aps_cvg.append(x.mean()) 

In [76]:
agg_aps_cvg

[0.9479820627802692,
 0.9426008968609866,
 0.9336322869955158,
 0.9246636771300448,
 0.9147982062780269,
 0.8941704035874439,
 0.8932735426008968,
 0.8681614349775785,
 0.863677130044843,
 0.8573991031390135,
 0.8484304932735427]

In [77]:
agg_aps_size = []
for x in aps_agg_size:
    agg_aps_size.append(x.mean()) 
agg_aps_size

[1.6188340807174888,
 1.5865470852017938,
 1.5461883408071748,
 1.495964125560538,
 1.4385650224215247,
 1.3551569506726457,
 1.3300448430493275,
 1.240358744394619,
 1.209865470852018,
 1.1829596412556054,
 1.1497757847533632]